In [1]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
root.addHandler(handler)

import os
os.environ["LITELLM_LOG"] = "DEBUG"
import litellm
litellm.set_verbose=True

02:43:23 - LiteLLM:DEBUG: utils.py:137 - Exception import enterprise features No module named 'litellm.proxy.enterprise'


In [2]:
import phoenix as px

from openinference.semconv.resource import ResourceAttributes
from openinference.instrumentation.dspy import DSPyInstrumentor
# from clank.so.openinference.semconv.resource import ResourceAttributes
# from clank.so-openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.trace import SpanAttributes

endpoint = "http://127.0.0.1:6006/v1/traces"
# resource = Resource(attributes={})
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'Span-test'
})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

2024-06-25 02:43:24,438 - phoenix.config - INFO - 📋 Ensuring phoenix working directory: C:\Users\Felix\.phoenix
2024-06-25 02:43:24,451 - phoenix.inferences.inferences - INFO - Dataset: phoenix_dataset_798a7317-347f-4086-a79d-d3ee8b96738d initialized


In [3]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.modeling import JSONBackend, TextBackend

In [4]:
gsm8k = GSM8K()

# backend = JSONBackend(model="ollama/llama3:70b", api_base="http://localhost:11434", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9})
# backend = JSONBackend(model="ollama/llama3:70b", api_base="http://localhost:11434", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9, "response_format": {"type": "json_object"}})


# backend = TextBackend((model="ollama/llama3:70b", api_base="http://localhost:11434", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9})
backend = TextBackend(model="ollama/llama3:70b", params={"max_tokens": 500, "temperature": 0.3, "num_retries": 5, "repeat_penalty": 1.2, "top_p": 0.9, "response_format": {"type": "json_object"}})

dspy.settings.configure(backend=backend)

trainset, devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 31992.22it/s]


In [5]:
NUM_THREADS = 4
evaluate = Evaluate(devset=devset[:], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)

In [6]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [7]:
RUN_FROM_SCRATCH = True

if RUN_FROM_SCRATCH:
    config = dict(max_bootstrapped_demos=3, max_labeled_demos=3, num_candidate_programs=3, num_threads=NUM_THREADS)
    teleprompter = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)
    cot_bs = teleprompter.compile(CoT(), trainset=trainset, valset=devset)
    # cot_bs.save('turbo_8_8_10_gsm8k_200_300.json')
else:
    cot_bs = CoT()
    cot_bs.load('turbo_8_8_10_gsm8k_200_300.json')

2024-06-25 02:43:38,852 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:43:38.852870Z [info     ] Going to sample between        [dspy.teleprompt.random_search] filename=random_search.py lineno=58 positional_args=(1, 'and', 3, 'traces per predictor.')
2024-06-25 02:43:38,853 - dspy.teleprompt.random_search - INFO - 2024-06-25T06:43:38.853869Z [info     ] Will attempt to train          [dspy.teleprompt.random_search] filename=random_search.py lineno=61 positional_args=(3, 'candidate sets.')
Model Kwargs: Model Kwargs: Model Kwargs: Model Kwargs:     {'temperature': 0.3, 'max_tokens': 500, 'top_p': 0.9, 'frequency_penalty': 0, 'num_retries': 5, 'repeat_penalty': 1.2, 'response_format': {'type': 'json_object'}, 'messages': [{'role': 'user', 'content': "Given the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\

  0%|          | 0/10 [00:00<?, ?it/s]

{'temperature': 0.3, 'max_tokens': 500, 'top_p': 0.9, 'frequency_penalty': 0, 'num_retries': 5, 'repeat_penalty': 1.2, 'response_format': {'type': 'json_object'}, 'messages': [{'role': 'user', 'content': "Given the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\n\nReasoning: Let's think step by step in order to"}]}{'temperature': 0.3, 'max_tokens': 500, 'top_p': 0.9, 'frequency_penalty': 0, 'num_retries': 5, 'repeat_penalty': 1.2, 'response_format': {'type': 'json_object'}, 'message

02:43:38 - LiteLLM:WARNING: utils.py:338 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.



2024-06-25 02:43:38,907 - LiteLLM - WARNING - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

Average Metric: 0.0 / 8  (0.0):  80%|████████  | 8/10 [03:26<00:28, 14.39s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Logging Details: logger_fn - None | callable(logger_fn) - False


Request to litellm:
litellm.completion(model='ollama/llama3:70b', api_key=None, api_base=None, temperature=0.3, max_tokens=500, top_p=0.9, frequency_penalty=0, repeat_penalty=1.2, response_format={'type': 'json_object'}, messages=[{'role': 'user', 'content': "Given the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nQuestion: Martha's cat catches 3 rats and 7 birds. Cara's cat catches 3 less than five times as many animals as Martha's cat. How many animals does Cara's cat catch?\n\nReasoning: Let's think step by step in order to"}], litellm_ca

Average Metric: 0.0 / 9  (0.0):  90%|█████████ | 9/10 [04:15<00:25, 25.21s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Logging Details: logger_fn - None | callable(logger_fn) - False


APIConnectionError: litellm.APIConnectionError: 'name'
Traceback (most recent call last):
  File "c:\llm\env-phoenix-dspy-2_5\Lib\site-packages\litellm\main.py", line 2362, in completion
    generator = ollama.get_ollama_response(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\llm\env-phoenix-dspy-2_5\Lib\site-packages\litellm\llms\ollama.py", line 283, in get_ollama_response
    "name": function_call["name"],
            ~~~~~~~~~~~~~^^^^^^^^
KeyError: 'name'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\llm\env-phoenix-dspy-2_5\Lib\site-packages\litellm\utils.py", line 851, in wrapper
    result = original_function(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\llm\env-phoenix-dspy-2_5\Lib\site-packages\litellm\main.py", line 2585, in completion
    raise exception_type(
          ^^^^^^^^^^^^^^^
  File "c:\llm\env-phoenix-dspy-2_5\Lib\site-packages\litellm\utils.py", line 7461,

In [ ]:
evaluate(cot_bs, devset=devset[:])

In [ ]:
print(backend.history[-1].prompt.to_str())